In [ ]:
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *

topdir='/sandbox/lsst/lsst/GUI/'
thedir=topdir+'testdata'
%cd $thedir


In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

In [ ]:
ct_matrix = ones([16,16])
# Old Matrix
"""
ct_matrix = array([[1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0],
                    [1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0],
                    [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0],
                    [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0]])
"""
segmentmap = [108,109,110,111,112,113,114,115,107,106,105,104,103,102,101,100]

segmentcoords = [[50,1000],[50,1000],[50,1000],[50,1000],[50,1000],[50,1075],[50,1100],[40,1100],[50,1100],[50,1100],[50,1080],[50,1050],[50,1040],[50,1020],[50,1010],[50,990]]

attackradius = 5 # This the radius around the segmentcoords coordinates where we look
xcoomin = 10
xcoomax = 100
ycoomin = 500
ycoomax = 1500

for attacksegment in range(16):
    fitsfilename = glob.glob(thedir+'/114-04_target_light_%d_20160621??????.fits'%segmentmap[attacksegment])[0]
    print fitsfilename
    attackhdr=pyfits.getheader(fitsfilename,attacksegment+1)
    attackextname = attackhdr['EXTNAME']
    attackimg=pyfits.getdata(fitsfilename,attackextname) 
    attackosimg=remove_overscan_xy(attackimg,509,542,2000,2022)
    attackbackground = median(attackosimg[ycoomin:ycoomax,xcoomin:xcoomax])
    for basesegment in range(16):
        if basesegment == attacksegment or ct_matrix[basesegment,attacksegment] < 0.1:
            continue
        basehdr=pyfits.getheader(fitsfilename,basesegment+1)
        baseextname = basehdr['EXTNAME']

        baseimg=pyfits.getdata(fitsfilename,baseextname) 
        baseosimg=remove_overscan_xy(baseimg,509,542,2000,2022)
        basebackground = median(baseosimg[ycoomin:ycoomax,xcoomin:xcoomax])
        ratio = []
    
        for i in range(segmentcoords[attacksegment][0]-attackradius, segmentcoords[attacksegment][0]+attackradius):
            for j in range(segmentcoords[attacksegment][1]-attackradius, segmentcoords[attacksegment][1]+attackradius):
                #print i, j, baseosimg[j,i], basebackground, attackosimg[j,i], attackbackground
                r = (baseosimg[j,i]-basebackground) / (attackosimg[j,i]-attackbackground)
                ratio.append(r)
        medratio = median(array(ratio))
        if len(ratio) == 0 or abs(medratio) < 1.0E-4:
            crosstalk_element = 0.0
        else:
            crosstalk_element = median(array(ratio))
        ct_matrix[basesegment,attacksegment] = crosstalk_element
        print "Base %s, Attack %s, ratio = %.4f"%(baseextname,attackextname,crosstalk_element)

        #hist(ratio)
        #show()
     
     
print "[",
for i in range(16):
    print "[",
    for j in range(16):
        if j < 15:
            print "%+.4f,"%ct_matrix[i,j],
        else:
            print "%+.4f],"%ct_matrix[i,j]
print "]"

"""

print "Nearest = %.4f"%(min(array(ratio)))
print "Next Nearest = %.4f"%(min(array(ratio)))
subplot(1,2,1)
hist(nearest_ratio)
subplot(1,2,2)
hist(next_nearest_ratio)
show()
"""
        
